# Book Recommendation System

**Team members: Deepak Maran, Kewei Liu, Rakshita Nagalla, Xiaohui Guo**  

Part I of the project for E4571 Personalization: Theory & Application.  

## Introduction
Book crossing is a website that enables free sharing of books, either with friends or with complete strangers by wild-releasing books in public places. They offer optional paid membership for new members that includes features like book recommendation. These recommendations direct to the  Better World Books website which is an online book-selling portal.  

In this context, better top-k recommendations generate revenue to Bookcrossing in two ways:  

1. Better recommendations translate to purchases at World Books and Bookcrossings gets a share.  

2. Personalized recommendations leads to user satisfaction resulting in retention of users and brand value. This contributes to revenue through paid memberships as new users would opt for the optional membership and existing users would continue to pay to retain the membership.  

Thus, our business objective is to generate k recommendations(top-k recommendation) for users logged into bookcrossings website.

## Dataset Source
Book crossing is a website that enables free sharing of books, either with friends or with complete strangers by wild-releasing books in public places. The data we are going to use is from [here](http://www2.informatik.uni-freiburg.de/~cziegler/BX/).  

There are three datasets.   
The BX-Users.csv contains information about 278,858 users.   
The BX-Books.csv contains information about 271,360 books.   
And the BX-Book-Rating contains information about 1,149,780 ratings, from 105,283 users and 340,556 items.  

The ratings dataset contains explicit feedbacks where users explicitly rate books on a scale of 1-10. We assume that this ratings data is a proxy of the actual preference of the user. 

## Recommendation Task
We recommend books to the user if the book’s estimated rating $\hat{r}_{ui}$ is greater than the threshold, and if it is among the k highest estimated ratings. Also, unlike some applications like music recommendation, users are not usually interested in repurchasing the books. We also take this into account while predicting and do not recommend book also seen in the train set.


## Evaluation metrics

### Accuracy
Since our objective is to recommend $k$ items to the user, evaluation metrics like RMSE and MAE which measure accuracy of rating prediction are not very indicative of the end-user experience. To satisfy our business objective, we are are more concerned about the precision at the top. Hence we choose the following evaluation metrics:  

- **Precision**:   
    Precision measures the proportion of recommended items that are actually relevant and is directly related to our business objective. We define an item as being relevant to a user, if he gave it a rating of 7 or above. We empirically chose the threshold value to be 7 based on the ratings distribution.  

- **Recall**:   
    Recall indicates the proportion of relevant items that are in the recommended list. We consider recall also because it provides information on the expected performance of the recommender system regarding the unknown complete data (i.e., all books), which exactly is what is experienced by the user. If we assume that relevant ratings are missing at random (while allowing all other rating values to be missing not at random), then Recall can be estimated without bias from observed MNAR data.  

- **F-score**:   
    F-score summarizes the trade-off between precision and recall.  
$$F\, score = \frac{2 * Precision * Recall}{Precision + Recall}$$
  
- **NDCG**:   
    In our case, the ranking between 1st and 2nd ranked books is more important than those at the end of the list. Moreover, unlike in precision and recall, NCDG does not require relevance to be binary, allowing the user’s preference of the book to expressed in multiple levels of relevance. In order to incorporate these idea, we use the normalized discounted cumulative gain to evaluation our recommendation lists.  
    
    $$ DCG = \frac{1}{m}\sum^{m}_{u=1}\sum_{j\in I_{u}, v_{j}\leq L} \frac{2^{rel_{uj}}-1}{log_{2}(v_{j}+1)} $$
    
    $$NDCG= \frac{DCG}{IDCG}$$
    
    where IDCG is the ideal DCG, computed using the ranking of actual rating.
    Note that, we choose $rel_{u,j}$ to be the actual rating of the user $u$ gave to item $j$.

We also plot the precision-recall curves for all the three algorithms for different values of $k$.

### Coverage
For a recommender system to be accurate, the recommendations should not only be accurate but also have high coverage, i.e., it should be able to recommend a good proportion of items. We look at two types of coverage:  

- **User-space Coverage**:  
    It indicates the percentage of users for whom the recommendation system is able to recommend at least k books.  
    
- **Item-space Coverage(Catalog Coverage)**:  
    It is specifically suited for recommendation lists. It is defined as the fraction of items that are recommended to at least one user.  
    
    $$CC = \frac{|\bigcup\limits_{u=1}^{m}T_u|}{total\,\,no.\,of\,items}$$  
    
    where $T_{u}$ is the recommendation list for user $u$.

## Cleaning the data
The original data contains ratings of value 0, which indicate implicit feedback. Since we will only use explicit feedbacks, we first filter out the ratings that are between 1 to 10.  

Since the website provides three dataset, containing information about users, books, and ratings, we filter out ratings whose corresponding user and book exist in the users and books dataset.  

To check whether the dataset is sparse or dense, we compute the density of the dataset. The density of a dataset means the percentage of user-book pairs that have a rating. It is calculated as :

$$Density = \frac{no.\,of\,ratings}{no.\,of\,users * no.\,of\,items}$$

The density of the remaining data is 0.0038%. So the dataset is very sparse and we need to perform a top-10 recommendation, we only work with items that were rated at least 10 times and users who have rated at least 20 items. Then the sparsity of the dataset becomes 0.8143%. At this stage, the dataset contain 1,117 users and 5,356 items. 

## Evaluation Setup
We put aside 22.9% of the data for testing. This test data was formed by selecting 10 random ratings for each user in the dataset. This ensures that there are a fair number of ratings for recommending top-10 books to the user.  

The remaining data forms the training data. The training data will then be splitted into train and validation sets using a 4-fold cross-validation and hyperparameters were tuned using those datasets.   

Once the tuning is complete, we train on the whole training data and predict on the held-out test data(Note that this test set was not used in hyperparameter training to ensure unbiased evaluation of the algorithms).

## Model 1: Neighborhood-based Collaborative Filtering

### k-Nearest Neighbor

### 1. Method Description
We begin by using the k-nearest neighbour method due to its simplicity and intuitiveness. The following variations were tried with kNN:   

1. Basic kNN, kNN with mean, KNN with Z-score  
2. Similarity: Cosine, Pearson, Pearson_baseline  
3. User-based, item-based  

The approach of taking item-based similarity is seen to perform better than user-based similarity. The search for users in a large user population is the bottleneck in user-based methods. This is overcome by instead exploring relationships between items first which are smaller in number.
A basic item-based kNN model with the Pearson similarity is seen to work best for our dataset, where the prediction is set as:

$$ \hat{r}_{ui} = \frac{\sum_{j\in N^{k}_{u}(i)}sim(i,j)\cdot r_{uj}}{\sum_{j\in N^{k}_{u}(j)}sim(i,j)}$$  

where $\hat{r}_{ui}$ is the predicted rating by user $u$ on item $i$, $r_{uj}$ is the actual rating by user $u$ on item $j$, and $sim(i,j)$ is the similarity between items $i$ and $j$ given by the Pearson co-efficient.

Other variations of the kNN model were tried to check if they could improve our evaluation metrics. The kNN model with mean offset was tried to correct possible offsets in the mean ratings by individual users or on individual items. The kNN model with Z-score correction was used to correct for the mean and standard deviation of the ratings. These variations did not give any improvements, and the basic kNN model was found to work best with respect to the evaluation metrics we want to optimize for.  

Different similarity metrics were tried with all the above variations. Figure 1 below shows some of the best performing kNN variations that were tried.  



### 2. Hyper-parameter Tuning
The number of neighbours $k$ is the hyperparameter to be tuned to maximize our evaluation metric (F-score) on the cross-validation set.  

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/kNN_tuning.png?raw=true">

Figure 1 shows the overall recommendation accuracy of the item-based algorithms does tend to improve marginally as we increase the value of $k$.  

However, this increase is not very significant. This is particularly important since small values of $k$ lead to fast recommendation rates (i.e., low computational requirements).  

Hence, we choose the value of $k$ where the evaluation metric saturates as the optimal value of $k$.  

Both NDCG and F-score saturate as $k=5$, which is the optimal choice for training our dataset.  

The kNN model may not be the best model for a sparse dataset like the Bookcrossings dataset, so model-based approaches are explored to see if they can improve our accuracy and coverage metrics.

## Model 2: Model-based Collaborative Filtering

### SVD Matrix Factorization

### 1. Model Description
We use the SVD-based Matrix factorization technique to recommend books to users. While SVD is not best-suited for sparse matrices, we choose this model due to its simplicity and effectiveness when using explicit data. Other optimization methods like ALS are not very effective in case of large explicit ratings matrices.   

Instead of assuming that a user $u$’s rating for item $i$ can be described simply by the dot product of the user and item latent vectors, we will consider that each user and book can have a bias term associated with them. The rationale is that certain users might not be very critical and tend to rate all books highly, or certain books may tend to always have low ratings. We also include a global bias term. These biases are learnt by the model. So for this model, the prediction $\hat{r}_{ui}$ is set as:  

$$ \hat{r}_{ui} = \mu + b_{u} + b_{i} + q^{T}_{i}p_{u} $$ 

If user $u$ is known, then the bias $b_{u}$ and the factors $p_{u}$ are assumed to be zero. The same applies for item $i$ with $b_{i}$ and $q_{i}$.  

We observe that our ratings matrix is very sparse even after the basic cleaning (with a density of 0.814%). Since the observed ratings are so few, the model is prone to overfit. We overcome this problem by adding regularization terms to the objective function.  

Hence, our goal is to minimize the following regularized squared error:  

$$ \sum_{r_{ui} \in R_{train}} (r_{ui}-\hat{r}_{ui})^{2} + \lambda(b_{i}^2 + b_{u}^2 + ||q_{i}||^2 + ||p_{u}||^2) $$

The minimization is performed by a very straightforward stochastic gradient descent:  

\begin{equation}
\begin{array}{rcl}
b_{u}&\leftarrow&b_{u} + \gamma(e_{ui} - \lambda b_{u})\\
b_{i}&\leftarrow&b_{i} + \gamma(e_{ui} - \lambda b_{i})\\
p_{u}&\leftarrow&p_{u} + \gamma(e_{ui}\cdot q_{i} - \lambda p_{u})\\
q_{i}&\leftarrow&q_{i} + \gamma(e_{ui}\cdot p_{u} - \lambda q_{i})\\
\end{array}
\end{equation}

### 2. Hyperparameter Tuning
For tuning hyperparameters, we need to split the training data into to training set and validation set. Test data is not used for tuning. We use training set to train models with different hyperparameters and use the validation set to compute accuracy, which is measure using F-score and NDCG.

We use cross validation to split the training data and compute average accuracy, which is a better evaluation than splitting the data only once and compute one accuracy. We use 4-fold cross validation. That is, the training set is randomly partitioned into 4 equal size subsamples, use each subsample as validation set and use rest of the data as training set. Repeat the process 4 times and compute accuracy for each time. Then we can get the average accuracy of the 4 accuraies.  

For our SVD-MF model, there are three hyperparameters, the number of latent factors, learning rate, and regularization term ($\lambda>0$).  


<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/factors.png?raw=true">

If we increase the number of factors, with learning rate and regularization term unchanged, we will introduce more parameters. Therefore the accuracy for training set will increase, as shown in the figures. Based on the evaluation of F-score, the validation set accuracy increases first because the model is changing from underfitting to a better model, then decreases because of overfitting. As for NDCG, the validation set accuracy keep decreasing as factors increases because of overfitting.  


<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/learningrates.png?raw=true">

For learning rate, if it is too small(e.g., 0.0001), the parameters may not reach the optimal solution. And if it is too large, the algorithm will not converage. Thus if learning rate is too small or too large, the model may not be well trained. So it cannot make good predictions, which can be shown by the accuracy on the validation set from the figures(learning rate = 0.0001, 0.001, 0.01, 0.02, 0.05).  

Now we first tune two parameters together, the number of latent factors and learning rate, while setting regularization term to default value 0.02. We compute accuracies for multiple value combinations of these two hyperparameters using grid search method with cross valiation. Then we choose the values of the number of latent factors and learning rate based on the accuracies. The larger the accuracy, the better the hyperparameter combination.  

Results:
- F-score on validation set:  

|no. of factors \ learning rate| 0.0001 | 0.001 |  0.01  | 0.02 | 0.05 |
|:---:|:---:|:---:|:---:|:---:|:---:|
|**5**|0.867|0.871|0.839|0.819|0.795|
|**10**|0.867|0.873|0.837|0.822|0.802|
|**15**|0.866|0.872|0.840|0.822|0.808|
|**20**|0.865|0.871|0.839|0.825|0.815|
|**50**|0.866|0.870|0.842|0.830|0.826|

- NDCG on validation set:  

|no. of factors \ learning rate| 0.0001 | 0.001 | 0.01 | 0.02 | 0.05 |
|:---:|:---:|:---:|:---:|:---:|:---:|
|**5**|0.800|0.820|0.822|0.807|0.801|
|**10**|0.794|0.818|0.820|0.808|0.802|
|**15**|0.789|0.818|0.821|0.820|0.805|
|**20**|0.782|0.817|0.820|0.813|0.807|
|**50**|0.777|0.813|0.816|0.811|0.815|

Based on the results, we can see that if factors $\in$ [5, 15], this hyperparameter will not affect the accuracy much. As for learning rate, combining the results of F-score and NDCG, a good choice might be 0.001. Therefore we choose the number of factors to be 10 and the learning rate to be 0.001, since the F-score of this combination is the largest, and the NDCG of this hyperparameter pair is close to the largest NDCG value in the results.  

Given the values of the number of latent factors and learning rate we have chosen, we tune on regularization term to avoid overfitting. We compute accuracies for different values of regularization term(0.02, 0.1, 0.5, 1, 1.5, 2) with cross validation. Then we choose the best regularization term based on the accuracies.   

Results:  
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/regs.png?raw=true">

From the figures above, we can see that for different regularization terms, the F-score on validation set does not change much. But the NDCG on the validation set first increases, which means larger regularization term reduces the overfitting problem, then decreases, which means now the model may have underfitting problem. Thus a good choice of regularization term might be 1, which gives the largest NDCG. 

Thus, we will use the number of latent factors = 10, learning rate = 0.01, and regularization term = 1.

### MAP Inference Coordinate Ascent Algorithm

### 1. Model Description
Probabilistic Matrix Factorization model performs well on the large, sparse, and very imbalanced dataset. What’s more,  probabilistic algorithms scale linearly with the number of observations. So we explored the probabilistic matrix factorization approach. The following is the MAP inference coordinate ascent algorithm.  

Input: An incomplete ratings matrix $M$, as indexed by th eset $\Omega$. Rank = $d$.  

Output: $N_{1}$ user locations, $u_{i} \in R^{d}$, and $N_{2}$ object locations, $v_{j} \in R^{d}$.  

Initialize each $v_{j}$ to normal distribution.  

for each iteration do:  
- for $i = 1, \dots, N_{1}$ update user location  
    
    $$ u_{i} = (\lambda\sigma^2I+\sum_{j \in \Omega_{u_{j}}}v_{j}v^T_{j})^{-1}(\sum_{j \in \Omega_{u_{i}}}M{ij}v_{j})$$

- for  $j = 1, \dots, N_{2}$ update object location  
    
    $$ v_{i} = (\lambda\sigma^2I+\sum_{i \in \Omega_{v_{j}}}u_{i}u^T_{i})^{-1}(\sum_{i \in \Omega_{v_{j}}}M{ij}u_{i})$$

Predict that user $i$ rates object as $u^T_iv_j$ rounded to closest rating option.  

The goal is to find the User matrix $U$ and Item matrix $V$ to maximize the objective value. The following is the objective function.  

$$ L = - \sum_{(i,j)\in \Omega}\frac{1}{2\sigma^2}||M_{ij}-u^T_iv_j||^2 - \sum_{i=1}^{N_{1}}\frac{\lambda}{2}||u_{i}||^2 - \sum_{j=1}^{N_{2}}\frac{\lambda}{2}||v_{j}||^2 + constant$$

### 2. Model Training
For our data set, there are 1,117 unique users and 5,472 unique books. Set the rank d to be 10. It means that the shape of matrix $U$ would be 1,117 x 10, the shape of matrix $V$ is 10 x 5,472.  

Implement the algorithm for 10 times. For each time, set the iterations to be 100. Then we get the following graph showing the objective value as a function of the number of iterations. The objective value is converging. Actually they all converge much earlier before the 100th iteration. So we can be sure the model is very well trained. 

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/Coordinate%20Ascent%2003%20Plot%20Objective%20value.png?raw=true">

Calculated the objective function value on each run, we got the following graph.  

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/Coordinate%20Ascent%2004%20Select%20the%20best%20run.png?raw=true">

The third run had the highest objective function value. So we used the matrix $U$ and matrix $V$ created by the final iteration, which is the 100th iteration, of the third run. We used these two matrices to calculate the predicted rating given by a customer for a book. For example, the rating from the ith audience for the jth book, which is $M_{ij}$, is equal to the dot product of $U_i$ and $V_j$. $U_i$ is the ith row of the matrix $U$ and $V_j$ is the jth column of the matrix $V$ respectively.   

Used the trained model to make prediction on the test data. The prediction precision is 0.8522, which is better than the Knn prediction precision 0.8155. The NDCG 0.8286 is also higher than that of kNN 0.8013. But in terms of the metrics of coverage and F-score, the Probabilistic Matrix Factorization model didn’t perform as well as kNN. What’s more, the main drawback of probabilistic matrix factorization approach is that it is computationally expensive. So kept on exploring other methods in matrix factorization.

## Evaluation

### Evaluation Metrics
We compute the evaluation metrics for kNN, SVD-MF, MAP inference coordinate ascent, and a baseline model. The baseline model predicts a random rating based on the distribution of the training set, which is assumed to be normal.  

In the following table, we report both the biased estimates of metrics on train data and unbiased estimate of metrics on held-out test data set. These metrics were calculated for a recommendation list of size 10.  

| |Baseline|KNN|SVD-MF|MAP Inference Coordinate Ascent|
|:---:|:---:|:---:|:---:|:---:|
|Train NDCG|0.5789|0.9186|0.7590|0.9994|
|Test NDCG|0.8013|0.8247|0.8538|0.8286|
|Train F-score|0.5807|0.6966|0.6720|0.7034|
|Test F-score|0.7542|0.8702|0.9015|0.6331|
|Train Precision|0.8200|0.9818|0.8937|1.0000|
|Test Precision|0.8155|0.8226|0.8236|0.8522|
|Train recall|0.4495|0.5398|0.5384|0.5425|
|Test recall|0.7014|0.9237|0.9955|0.5036|
|Train User Coverage|0.7681|0.8111|0.9893|0.7645|
|Test User Coverage|0.0304|0.6079|0.9687|0.0116|
|Train Item Coverage|0.7493|0.7489|0.5819|0.6454|
|Test Item Coverage|0.8591|0.9797|0.9961|0.5481|

From the above table, we observe that all the models perform better than the random predictions model that we used as  a baseline. Moreover, Matrix factorization using SVD performs consistently better than the kNN model. Given the extreme sparsity of the data set, these results makes sense. Memory based methods like kNN overfit to the sparse data we have, and do not generalize at all. SVD, on the other hand, overcomes this problem by introducing the regularization term in the objective function. It is also helpful to remember that we found the regularization parameter ($\lambda$) of value 1 to be optimum. This relatively higher value of $\lambda$ is also indicative of the high sparsity in the data.

### Precision Recall Curves
The metrics reported in the above table assume a recommendation list size of 10. However, the optimal value of recommended list is not known. If the list is too small, the algorithm might miss relevant items and larger lists might lead to spurious recommendations. In order to get a better sense of how the algorithms perform for varying lengths of recommendation lists, we plot the precision recall curve for $k$ varying from 1 to 10 (Recommending more than 10 books might be too tasking for the user).   

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/precision-recall-curve.png?raw=true">


We observe that the precision-recall(PR) curve for SVD strictly dominates the PR curve for kNN, indicating that SVD performs better than kNN while recommending lists containing 1 to 10 items.

## Scalability
We now attempt to evaluate the scalability of the model to understand the effect of model size on the performance and running time of the models explored. To this end, we randomly sample 40%, 20%, 10%, 5% and 1% of the total data available.  

We also filter these random samples using the same way we filtered the data initially, i.e, all users that have rated less than 20 items are filtered (Due to the limitation of data size when we sample only 1% or 5% of the data, we do not filter out items as we did before).  

We then evaluate our models on these varying sample sizes using around 80% data for training and 20% for testing.

### 1. Accuracy vs Size
<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/accuracysize.png?raw=true">

The first figure above illustrates how F-score varies as the sample size increases. We observe that performance of both k-NN and svd models as measured by the F-score metric decreases as sample size increases. This could be attributed to the fact that the subsampling inevitably makes the sample denser. Because the way we clean data somehow will filter out more active users(i.e., users that rated lots of books). So if we randomly sample the data, those active users may still satisfy our filtering condition(i.e., rated at least 20 books), but those less active users are more likely to be cleaned. Hense as the sample size becomes smaller, the proportion of active user becomes larger, thus more dense. The takeaway from this section is that SVD Matrix factorization models perform better than memory based models as the ratings matrix becomes sparser.  

The second figure above illustrates how NDCG varies as the sample size increases. Overall, the value of NDCG almost remains constant as the sample size varies. The small variation in NDCG may result from the random sampling.  

It is also worth noting that while SVD seems to perform better than kNN across all sample sizes, the gap is much lesser for smaller sample sizes(i.e., denser ratings matrix). This indicates that for very sparse datasets like ours, Matrix factorization based methods like SVD perform better than memory based methods like kNN.

### 2. Run-time vs Size
The following figure indicates how the running time of the kNN and SVD Matrix Factorization algorithms varies as the model size increases. Note that running time here indicates time taken to train the model on training data and to test it on the test data.  

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/runtimesize1.png?raw=true">

We observe that as the sample size increases, the the running time of the item-based nearest neighbors algorithm rapidly increases. This is expected as the number of items increases with the sample size and so does the time taken by the kNN algorithm to compute k nearest items.  

Now let us take a closer look at the plot of running time of SVD algorithm as shown in the figure below.  

<img src="https://github.com/Kewei-Liu/E4571_Personalization_Project-Part_I/blob/master/image/runtimesize2.png?raw=true">

We observe that running time of the SVD algorithm also increases with time but much slower than the k-NN algorithm.

## Conclusion
The biggest challenge in generating top-10 recommendations using the bookcrossing dataset is the extreme sparsity. SVD performs reasonably well both in terms of accuracy and coverage at top-k. Its running time and accuracy also scales well with the sample size in comparison to memory based methods. Another MF technique that can also be explored is the Non-Negative Matrix Factorization technique due to the non-negative nature of the ratings, especially for the greater interpretability. Moreover, online evaluation metrics like click-through rate should be used to evaluate to actual efficacy of the recommender system in terms of achieving the business objective.  

In this study, we limit ourselves to collaborative filtering methods. However the information about the books can be easily obtained using the ISBN number of the book, to overcome the cold start problem when the new user becomes a member, or when a new book is released. A hybrid of content based and collaborative filtering methods can be used to generate more relevant recommendations.  

However, for the recommendations to translate to purchases or higher user satisfaction, we need to evaluate our recommender system in terms of how diverse and serendipitous the recommendations are. Also, since bookcrossing provides information about location of the user, this information can be used to notify the user of book tours near his location. This would help improve the effectiveness of the wild releases that Book-crossings already conducts in public spaces.

## Reference

https://www.networkworld.com/article/2322697/software/bookcrossing-com--chapter-2.html  
http://www.bookcrossing.com/faqs  
https://www.betterworldbooks.com/go/bwb-bookcrossing  
https://pdfs.semanticscholar.org/ac74/6980f8f2a6817f3f9438577ccc0de0c0a867.pdf  
